In [2]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [3]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
#from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3
import ee

In [4]:
# service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, 'google_cred.json')
# import ee
# ee.Initialize(credentials)

In [5]:
import ee
ee.Authenticate()

Enter verification code: 4/1AdQt8qhgy0RbK03eeaj9KiKkHdIDwytCbAxwwU9Db61nVbv0wEzMTVxxzdk

Successfully saved authorization token.


In [6]:
ee.Initialize()

In [7]:
# define exceedence threshold
THRESHOLD = 5.0

In [8]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [9]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [10]:
pm25_2020 = ee.Image("projects/wri-datalab/Cities4Forests/acag_pm25-surface")
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.eq('year', 2020)).mosaic()

In [11]:
exceedance_2020 = pm25_2020.gte(THRESHOLD)

In [12]:
threshold_percentage = pm25_2020.multiply(1.0/THRESHOLD).multiply(100.0)

In [13]:
exceedance_pop = worldpop_2020.mask(exceedance_2020)

In [14]:
def exceedancepop(geom):
    local_exceedance_pop = exceedance_pop.reduceRegion(ee.Reducer.sum(), geom, 100, 'epsg:4326').getInfo()
    local_pop = worldpop_2020.reduceRegion(ee.Reducer.sum(), geom, 100, 'epsg:4326').getInfo()
    return local_exceedance_pop['population'] / local_pop['population']

# Only AOI level

In [15]:
pm25_indicator = pd.DataFrame() 

In [16]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    for boundary_id in [boundary_id_aoi, boundary_id_unit]:
        if boundary_id[-3:] != 'nan':
            print("\n geo_name: " + boundary_id)

            # process aoi level ------
            print("\n boundary_id_aoi: " + boundary_id)
            # read boundaries
            boundary_path = aws_s3_dir + '/boundaries/v_0/boundary-'+ boundary_id + '.geojson'
            boundary_geo = requests.get(boundary_path).json()
            boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)

            df = geemap.ee_to_pandas(boundary_geo_ee)
            df['exceedance_popfraction'] = exceedancepop(boundary_geo_ee)
            df['max_conc'] = threshold_percentage.reduceRegion(ee.Reducer.max(), boundary_geo_ee.geometry(), 100, 'epsg:4326').getInfo()['b1']
    
            pm25_indicator = pd.concat([pm25_indicator, df])

0

 geo_name: BRA-Salvador-ADM4union

 boundary_id_aoi: BRA-Salvador-ADM4union

 geo_name: BRA-Salvador-ADM4

 boundary_id_aoi: BRA-Salvador-ADM4
1

 geo_name: COD-Bukavu-ADM3union

 boundary_id_aoi: COD-Bukavu-ADM3union

 geo_name: COD-Bukavu-ADM3

 boundary_id_aoi: COD-Bukavu-ADM3
2

 geo_name: COD-Uvira-ADM3union

 boundary_id_aoi: COD-Uvira-ADM3union

 geo_name: COD-Uvira-ADM3

 boundary_id_aoi: COD-Uvira-ADM3
3

 geo_name: COG-Brazzaville-ADM4union

 boundary_id_aoi: COG-Brazzaville-ADM4union

 geo_name: COG-Brazzaville-ADM4

 boundary_id_aoi: COG-Brazzaville-ADM4
4

 geo_name: COL-Barranquilla-ADM4union

 boundary_id_aoi: COL-Barranquilla-ADM4union

 geo_name: COL-Barranquilla-ADM4

 boundary_id_aoi: COL-Barranquilla-ADM4
5

 geo_name: ETH-Addis_Ababa-ADM4union

 boundary_id_aoi: ETH-Addis_Ababa-ADM4union

 geo_name: ETH-Addis_Ababa-ADM4

 boundary_id_aoi: ETH-Addis_Ababa-ADM4
6

 geo_name: ETH-Dire_Dawa-ADM3union

 boundary_id_aoi: ETH-Dire_Dawa-ADM3union

 geo_name: ETH-Dire_Da

In [18]:
# for i in range(0, len(boundary_georef)):
#     print(i)
#     geo_name = boundary_georef.loc[i, 'geo_name']
    
    
#     boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
#     boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
#     print("\n geo_name: "+boundary_id_aoi)
    
#     # process aoi level ------
#     print("\n boundary_id_aoi: "+boundary_id_aoi)
#     # read boundaries
#     boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
#     boundary_geo = requests.get(boundary_path).json()
#     boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
#     df = geemap.ee_to_pandas(boundary_geo_ee)
#     df['exceedance_popfraction'] = exceedancepop(boundary_geo_ee)
#     pm25_indicator = pd.concat([pm25_indicator, df])

In [17]:
pm25_indicator

,geo_parent_name,geo_level,creation_date,geo_id,geo_name,exceedance_popfraction,max_conc
0,BRA-Salvador,ADM4-union,2022-08-03,BRA-Salvador_ADM4-union_1,BRA-Salvador,0.887474,233.999996
0,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_1,Pituaçu,0.888715,233.999996
1,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_2,Patamares,0.888715,233.999996
2,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_3,Piatã,0.888715,233.999996
3,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_4,Boca do Rio,0.888715,233.999996
...,...,...,...,...,...,...,...
135,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_136,Burengo,1.000000,811.999969
136,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_137,Bwamazi,1.000000,811.999969
137,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_138,Kadahenda,1.000000,811.999969
138,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_139,Karwesero,1.000000,811.999969


In [21]:
cities_indicators_GRE_2_3 = pm25_indicator

In [22]:
cities_indicators_GRE_2_3 = cities_indicators_GRE_2_3.round({'max_conc': 0})
cities_indicators_GRE_2_3 = cities_indicators_GRE_2_3.rename(columns={"max_conc": "GRE_2_3_population_exposure_pm25"})
cities_indicators_GRE_2_3

,geo_parent_name,geo_level,creation_date,geo_id,geo_name,exceedance_popfraction,GRE_2_3_population_exposure_pm25
0,BRA-Salvador,ADM4-union,2022-08-03,BRA-Salvador_ADM4-union_1,BRA-Salvador,0.887474,234.0
0,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_1,Pituaçu,0.888715,234.0
1,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_2,Patamares,0.888715,234.0
2,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_3,Piatã,0.888715,234.0
3,BRA-Salvador,ADM4,2022-08-03,BRA-Salvador_ADM4_4,Boca do Rio,0.888715,234.0
...,...,...,...,...,...,...,...
135,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_136,Burengo,1.000000,812.0
136,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_137,Bwamazi,1.000000,812.0
137,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_138,Kadahenda,1.000000,812.0
138,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_139,Karwesero,1.000000,812.0


# Upload in aws s3

In [23]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [24]:
# upload to aws
key_data = 'data/indicators/dev/cities_indicators_GRE_2_3.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_GRE_2_3.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [25]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')